In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
from skfeature.function.similarity_based import fisher_score

result= fisher_score.fisher_score(X_sm_tl, y_sm_tl)
result

C:\Users\User\anaconda3\lib\site-packages\skfeature\utility\construct_W.py:194: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  class_idx_all = class_idx[:, np.newaxis] & class_idx[np.newaxis, :]


array([20, 17, 15,  2, 19,  7,  1,  3, 21, 18, 11, 10, 13,  0,  8,  4, 12,
       16, 14,  5,  9,  6])

In [8]:
k= 10
top_k_indices= result[:k]

feature_names= X.columns
fisher_df= pd.DataFrame({'Feature': feature_names,
                         'Fisher Score': [np.nan] * len(feature_names)})

for idx in range(len(result)):
    fisher_df.loc[result[idx], 'Fisher Score']= idx + 1
    
fisher_df= fisher_df.sort_values(by='Fisher Score')
fisher_df.head(10)

,Feature,Fisher Score
20,HBP,1.0
17,Triglycerides,2.0
15,Income,3.0
2,Cholesterol,4.0
19,Sleep Hours Per Day,5.0
7,Obesity,6.0
1,Sex,7.0
3,Heart Rate,8.0
21,LBP,9.0
18,Physical Activity Days Per Week,10.0


In [9]:
X_sm_tl_selected= X_sm_tl[:, top_k_indices]
X_sm_tl_test_selected= X_test[:, top_k_indices]

--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.64      0.42      0.51      1125
           1       0.36      0.57      0.44       628

    accuracy                           0.48      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.48      0.48      1753

Precision: 0.3551587301587302
Accuracy: 0.47518539646320596
AUC: 0.489661712668082


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.43      0.52      1125
           1       0.37      0.59      0.45       628

    accuracy                           0.49      1753
   macro avg       0.51      0.51      0.49      1753
weighted avg       0.55      0.49      0.49      1753

Precision: 0.3671259842519685
Accuracy: 0.4877353108956075
AUC: 0.5111967445152158


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best Precision Score: 0.592083389982306


In [13]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.66      0.39      0.49      1125
           1       0.37      0.65      0.47       628

    accuracy                           0.48      1753
   macro avg       0.52      0.52      0.48      1753
weighted avg       0.56      0.48      0.48      1753

Precision: 0.37158469945355194
Accuracy: 0.48088990302338847
AUC: 0.5162661004953999


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.66      0.54      0.59      1125
           1       0.38      0.50      0.43       628

    accuracy                           0.52      1753
   macro avg       0.52      0.52      0.51      1753
weighted avg       0.56      0.52      0.53      1753

Precision: 0.3759036144578313
Accuracy: 0.5242441528807759
AUC: 0.5161174805378628


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 36, 'weights': 'uniform'}
Best Precision Score: 0.5223206334134418


In [16]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.63      0.51      0.57      1125
           1       0.35      0.46      0.39       628

    accuracy                           0.49      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.34532374100719426
Accuracy: 0.4945807187678266
AUC: 0.4736220806794055


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.65      0.44      0.52      1125
           1       0.36      0.57      0.44       628

    accuracy                           0.49      1753
   macro avg       0.50      0.50      0.48      1753
weighted avg       0.54      0.49      0.49      1753

Precision: 0.3620689655172414
Accuracy: 0.48659440958357103
AUC: 0.5029794762915782


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.63      0.46      0.53      1125
           1       0.35      0.52      0.42       628

    accuracy                           0.48      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.48      0.49      1753

Precision: 0.34962805526036134
Accuracy: 0.4803194523673702
AUC: 0.4970226468506723


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.66      0.35      0.45      1125
           1       0.37      0.68      0.48       628

    accuracy                           0.47      1753
   macro avg       0.51      0.51      0.47      1753
weighted avg       0.56      0.47      0.46      1753

Precision: 0.3674698795180723
Accuracy: 0.46605818596691384
AUC: 0.4947813163481953


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best Precision Score: 0.6240579976321716


In [22]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.65      0.32      0.43      1125
           1       0.36      0.68      0.47       628

    accuracy                           0.45      1753
   macro avg       0.50      0.50      0.45      1753
weighted avg       0.54      0.45      0.45      1753

Precision: 0.3608074011774601
Accuracy: 0.452937820878494
AUC: 0.5016022646850673


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.61      0.30      0.41      1125
           1       0.34      0.66      0.45       628

    accuracy                           0.43      1753
   macro avg       0.48      0.48      0.43      1753
weighted avg       0.52      0.43      0.42      1753

Precision: 0.3447698744769874
Accuracy: 0.43011979463776384
AUC: 0.47650106157112526


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6473664575286173


In [26]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.63      0.21      0.32      1125
           1       0.36      0.78      0.49       628

    accuracy                           0.42      1753
   macro avg       0.49      0.50      0.40      1753
weighted avg       0.53      0.42      0.38      1753

Precision: 0.3556851311953353
Accuracy: 0.41585852823730746
AUC: 0.4722158527954706


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.69      0.18      0.29      1125
           1       0.37      0.85      0.51       628

    accuracy                           0.42      1753
   macro avg       0.53      0.52      0.40      1753
weighted avg       0.57      0.42      0.37      1753

Precision: 0.36813186813186816
Accuracy: 0.4227039361095265
AUC: 0.498864118895966


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6623437855335721


In [29]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.68      0.19      0.30      1125
           1       0.37      0.84      0.51       628

    accuracy                           0.42      1753
   macro avg       0.53      0.52      0.40      1753
weighted avg       0.57      0.42      0.37      1753

Precision: 0.36754507628294036
Accuracy: 0.42384483742156304
AUC: 0.4833503184713376


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.65      0.27      0.38      1125
           1       0.36      0.73      0.48       628

    accuracy                           0.44      1753
   macro avg       0.50      0.50      0.43      1753
weighted avg       0.54      0.44      0.42      1753

Precision: 0.359717868338558
Accuracy: 0.4375356531660011
AUC: 0.491242745930644


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.6581605997231321
              precision    recall  f1-score   support

           0       0.67      0.23      0.34      1125
           1       0.37      0.80      0.50       628

    accuracy                           0.43      1753
   macro avg       0.52      0.51      0.42      1753
weighted avg       0.56      0.43      0.40      1753

Precision: 0.36542669584245074
Accuracy: 0.4312606959498003
AUC: 0.4940863411181883


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1578
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.63      0.23      0.34      1125
           1       0.35      0.76      0.48       628

    accuracy                           0.42      1753
   macro avg       0.49      0.49      0.41      1753
weighted avg       0.53      0.42      0.39      1753

Precision: 0.3547423450336072
Accuracy: 0.41985168282943525
AUC: 0.47167728237791934


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000470 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1578
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.6560792339079039
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_lea

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.020521
0:	learn: 0.6912430	total: 144ms	remaining: 2m 23s
1:	learn: 0.6889372	total: 149ms	remaining: 1m 14s
2:	learn: 0.6876807	total: 155ms	remaining: 51.4s
3:	learn: 0.6848639	total: 161ms	remaining: 40s
4:	learn: 0.6836556	total: 165ms	remaining: 32.8s
5:	learn: 0.6820110	total: 172ms	remaining: 28.4s
6:	learn: 0.6786047	total: 177ms	remaining: 25s
7:	learn: 0.6755121	total: 182ms	remaining: 22.5s
8:	learn: 0.6735876	total: 189ms	remaining: 20.8s
9:	learn: 0.6721573	total: 194ms	remaining: 19.2s
10:	learn: 0.6703528	total: 199ms	remaining: 17.9s
11:	learn: 0.6687068	total: 206ms	remaining: 16.9s
12:	learn: 0.6675614	total: 211ms	remaining: 16s
13:	learn: 0.6667644	total: 216ms	remaining: 15.2s
14:	learn: 0.6655109	total: 221ms	remaining: 14.5s
15:	learn: 0.6643917	total: 227ms	remaining: 14s
16:	learn: 0.6639706	total: 233ms	remaining: 13.5s
17:	learn: 0.6629447	total: 239ms	remaining: 13s
18:	learn: 0.6614518	total: 245ms	remaining: 12.7s
19:	learn: 0.659450

160:	learn: 0.5686562	total: 1.02s	remaining: 5.34s
161:	learn: 0.5685304	total: 1.03s	remaining: 5.33s
162:	learn: 0.5683832	total: 1.03s	remaining: 5.31s
163:	learn: 0.5682797	total: 1.04s	remaining: 5.3s
164:	learn: 0.5679492	total: 1.04s	remaining: 5.29s
165:	learn: 0.5675380	total: 1.05s	remaining: 5.28s
166:	learn: 0.5672516	total: 1.06s	remaining: 5.27s
167:	learn: 0.5667584	total: 1.06s	remaining: 5.26s
168:	learn: 0.5661265	total: 1.07s	remaining: 5.25s
169:	learn: 0.5659305	total: 1.07s	remaining: 5.24s
170:	learn: 0.5657055	total: 1.08s	remaining: 5.23s
171:	learn: 0.5655934	total: 1.08s	remaining: 5.22s
172:	learn: 0.5652501	total: 1.09s	remaining: 5.21s
173:	learn: 0.5649523	total: 1.09s	remaining: 5.19s
174:	learn: 0.5646791	total: 1.1s	remaining: 5.18s
175:	learn: 0.5644565	total: 1.1s	remaining: 5.17s
176:	learn: 0.5643254	total: 1.11s	remaining: 5.16s
177:	learn: 0.5638499	total: 1.11s	remaining: 5.15s
178:	learn: 0.5636118	total: 1.12s	remaining: 5.14s
179:	learn: 0.5

328:	learn: 0.5223748	total: 1.91s	remaining: 3.9s
329:	learn: 0.5222010	total: 1.92s	remaining: 3.9s
330:	learn: 0.5220260	total: 1.93s	remaining: 3.89s
331:	learn: 0.5219252	total: 1.93s	remaining: 3.88s
332:	learn: 0.5216900	total: 1.93s	remaining: 3.87s
333:	learn: 0.5212760	total: 1.94s	remaining: 3.86s
334:	learn: 0.5209751	total: 1.94s	remaining: 3.85s
335:	learn: 0.5207480	total: 1.95s	remaining: 3.85s
336:	learn: 0.5205638	total: 1.95s	remaining: 3.84s
337:	learn: 0.5204484	total: 1.96s	remaining: 3.83s
338:	learn: 0.5198356	total: 1.96s	remaining: 3.82s
339:	learn: 0.5196538	total: 1.97s	remaining: 3.81s
340:	learn: 0.5194454	total: 1.97s	remaining: 3.81s
341:	learn: 0.5192049	total: 1.97s	remaining: 3.8s
342:	learn: 0.5190903	total: 1.98s	remaining: 3.79s
343:	learn: 0.5186201	total: 1.98s	remaining: 3.78s
344:	learn: 0.5183593	total: 1.99s	remaining: 3.77s
345:	learn: 0.5181656	total: 1.99s	remaining: 3.77s
346:	learn: 0.5179949	total: 2s	remaining: 3.76s
347:	learn: 0.5175

514:	learn: 0.4806661	total: 2.8s	remaining: 2.63s
515:	learn: 0.4805308	total: 2.8s	remaining: 2.63s
516:	learn: 0.4802706	total: 2.81s	remaining: 2.62s
517:	learn: 0.4800359	total: 2.81s	remaining: 2.62s
518:	learn: 0.4795265	total: 2.81s	remaining: 2.61s
519:	learn: 0.4792888	total: 2.82s	remaining: 2.6s
520:	learn: 0.4790620	total: 2.82s	remaining: 2.6s
521:	learn: 0.4788061	total: 2.83s	remaining: 2.59s
522:	learn: 0.4785225	total: 2.83s	remaining: 2.58s
523:	learn: 0.4783743	total: 2.84s	remaining: 2.58s
524:	learn: 0.4782307	total: 2.85s	remaining: 2.58s
525:	learn: 0.4780632	total: 2.85s	remaining: 2.57s
526:	learn: 0.4778368	total: 2.85s	remaining: 2.56s
527:	learn: 0.4775666	total: 2.86s	remaining: 2.55s
528:	learn: 0.4773346	total: 2.86s	remaining: 2.55s
529:	learn: 0.4771019	total: 2.87s	remaining: 2.54s
530:	learn: 0.4767857	total: 2.87s	remaining: 2.54s
531:	learn: 0.4765536	total: 2.88s	remaining: 2.53s
532:	learn: 0.4764148	total: 2.88s	remaining: 2.52s
533:	learn: 0.47

681:	learn: 0.4446467	total: 3.54s	remaining: 1.65s
682:	learn: 0.4444749	total: 3.55s	remaining: 1.65s
683:	learn: 0.4441601	total: 3.55s	remaining: 1.64s
684:	learn: 0.4439266	total: 3.56s	remaining: 1.64s
685:	learn: 0.4436862	total: 3.56s	remaining: 1.63s
686:	learn: 0.4434601	total: 3.56s	remaining: 1.62s
687:	learn: 0.4429974	total: 3.57s	remaining: 1.62s
688:	learn: 0.4428732	total: 3.57s	remaining: 1.61s
689:	learn: 0.4427026	total: 3.58s	remaining: 1.61s
690:	learn: 0.4424763	total: 3.58s	remaining: 1.6s
691:	learn: 0.4422070	total: 3.58s	remaining: 1.59s
692:	learn: 0.4419662	total: 3.59s	remaining: 1.59s
693:	learn: 0.4417775	total: 3.59s	remaining: 1.58s
694:	learn: 0.4416153	total: 3.6s	remaining: 1.58s
695:	learn: 0.4414356	total: 3.6s	remaining: 1.57s
696:	learn: 0.4412402	total: 3.6s	remaining: 1.57s
697:	learn: 0.4410418	total: 3.61s	remaining: 1.56s
698:	learn: 0.4408329	total: 3.61s	remaining: 1.55s
699:	learn: 0.4405710	total: 3.62s	remaining: 1.55s
700:	learn: 0.44

861:	learn: 0.4114893	total: 4.21s	remaining: 674ms
862:	learn: 0.4113769	total: 4.21s	remaining: 669ms
863:	learn: 0.4111979	total: 4.21s	remaining: 664ms
864:	learn: 0.4111087	total: 4.22s	remaining: 658ms
865:	learn: 0.4109114	total: 4.22s	remaining: 654ms
866:	learn: 0.4107557	total: 4.23s	remaining: 648ms
867:	learn: 0.4105825	total: 4.23s	remaining: 643ms
868:	learn: 0.4104719	total: 4.23s	remaining: 638ms
869:	learn: 0.4103502	total: 4.24s	remaining: 633ms
870:	learn: 0.4102049	total: 4.24s	remaining: 628ms
871:	learn: 0.4100254	total: 4.25s	remaining: 623ms
872:	learn: 0.4097940	total: 4.25s	remaining: 618ms
873:	learn: 0.4096577	total: 4.25s	remaining: 613ms
874:	learn: 0.4095531	total: 4.26s	remaining: 608ms
875:	learn: 0.4094356	total: 4.26s	remaining: 603ms
876:	learn: 0.4092626	total: 4.26s	remaining: 598ms
877:	learn: 0.4090804	total: 4.27s	remaining: 593ms
878:	learn: 0.4088413	total: 4.27s	remaining: 588ms
879:	learn: 0.4087419	total: 4.27s	remaining: 583ms
880:	learn: 

In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6799288	total: 1.53ms	remaining: 151ms
1:	learn: 0.6709938	total: 3.11ms	remaining: 152ms
2:	learn: 0.6623916	total: 4.13ms	remaining: 134ms
3:	learn: 0.6452313	total: 5.14ms	remaining: 123ms
4:	learn: 0.6334172	total: 6.11ms	remaining: 116ms
5:	learn: 0.6259072	total: 7.14ms	remaining: 112ms
6:	learn: 0.6238626	total: 8.11ms	remaining: 108ms
7:	learn: 0.6215881	total: 10.2ms	remaining: 117ms
8:	learn: 0.6049118	total: 12.3ms	remaining: 125ms
9:	learn: 0.5919000	total: 14.4ms	remaining: 129ms
10:	learn: 0.5806773	total: 15.5ms	remaining: 125ms
11:	learn: 0.5714856	total: 16.5ms	remaining: 121ms
12:	learn: 0.5636325	total: 17.5ms	remaining: 117ms
13:	learn: 0.5571945	total: 18.6ms	remaining: 114ms
14:	learn: 0.5554145	total: 19.5ms	remaining: 111ms
15:	learn: 0.5535603	total: 20.5ms	remaining: 108ms
16:	learn: 0.5456894	total: 21.6ms	remaining: 105ms
17:	learn: 0.5431913	total: 22.6ms	remaining: 103ms
18:	learn: 0

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
3         KNeighborsClassifier Precision:         0.375904
2          Tuned Decision Tree Precision:         0.371585
17                    CatBoost Precision:         0.369565
11            GradientBoosting Precision:         0.368132
12      Tuned GradientBoosting Precision:         0.367545
7                Random Forest Precision:         0.367470
1                Decision Tree Precision:         0.367126
14                   Tuned XGB Precision:         0.365427
5                   GaussianNB Precision:         0.362069
16                  Tuned LGBM Precision:         0.361357
8          Tuned Random Forest Precision:         0.360807
13                         XGB Precision:         0.359718
10              Tuned AdaBoost Precision:         0.355685
18              Tuned CatBoost Precision:         0.355590
0          Logistic Regression Precision:         0.355159
15                        LGBM Precision:         0.3547

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
3         KNeighborsClassifier Accuracy:        0.524244
4   Tuned KNeighborsClassifier Accuracy:        0.494581
1                Decision Tree Accuracy:        0.487735
5                   GaussianNB Accuracy:        0.486594
2          Tuned Decision Tree Accuracy:        0.480890
6                          SVM Accuracy:        0.480319
0          Logistic Regression Accuracy:        0.475185
7                Random Forest Accuracy:        0.466058
8          Tuned Random Forest Accuracy:        0.452938
13                         XGB Accuracy:        0.437536
14                   Tuned XGB Accuracy:        0.431261
9                     AdaBoost Accuracy:        0.430120
18              Tuned CatBoost Accuracy:        0.429549
16                  Tuned LGBM Accuracy:        0.427268
12      Tuned GradientBoosting Accuracy:        0.423845
11            GradientBoosting Accuracy:        0.422704
17                    CatBoost 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
2          Tuned Decision Tree AUC:   0.516266
3         KNeighborsClassifier AUC:   0.516117
1                Decision Tree AUC:   0.511197
5                   GaussianNB AUC:   0.502979
8          Tuned Random Forest AUC:   0.501602
11            GradientBoosting AUC:   0.498864
6                          SVM AUC:   0.497023
7                Random Forest AUC:   0.494781
14                   Tuned XGB AUC:   0.494086
13                         XGB AUC:   0.491243
0          Logistic Regression AUC:   0.489662
16                  Tuned LGBM AUC:   0.486289
12      Tuned GradientBoosting AUC:   0.483350
17                    CatBoost AUC:   0.480733
18              Tuned CatBoost AUC:   0.477977
9                     AdaBoost AUC:   0.476501
4   Tuned KNeighborsClassifier AUC:   0.473622
10              Tuned AdaBoost AUC:   0.472216
15                        LGBM AUC:   0.471677
